# Variational monte carlo simulation for Helium Atom

In [19]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [20]:
def wavefunc(r,a):
    r1 = np.linalg.norm(r[:,0:3],axis=1)
    r2 = np.linalg.norm(r[:,3:6],axis=1)
    r12 = np.linalg.norm(r[:,0:3]-r[:,3:6],axis=1)
    return np.exp(-2*r1) * np.exp(-2*r2) * np.exp(r12/(2*(1+a*r12)))
    
def updateWalker(r,N,a):
    dis = np.random.normal(0,d,(N,6)) #proposed displacement
    p = (wavefunc(r+dis,a)/wavefunc(r,a))**2
    rnd = np.random.rand(N)
    r += (rnd<p)[:,None]*dis
    
def EL(r,a):
    r1 = np.linalg.norm(r[:,0:3],axis=1)
    r2 = np.linalg.norm(r[:,3:6],axis=1)
    r12 = np.linalg.norm(r[:,0:3]-r[:,3:6],axis=1)
    dotpr = np.sum((r[:,0:3]/r1[:,None] - r[:,3:6]/r2[:,None]) * (r[:,0:3] - r[:,3:6]),axis=1)
    return -4 + dotpr/(r12*(1+a*r12)**2) - 1/(r12*(1+a*r12)**3) - 1/(4*(1+a*r12)**4) + 1/r12

def updateVarParams(R,EL,a,gamma):
    R12 = np.linalg.norm(R[:,0:3]-R[:,3:6],axis=1)
    dlnpsida = -R12**2 / (2*(1+a*R12)**2)
    dEda = 2 * (np.mean(EL*dlnpsida)-np.mean(EL)*np.mean(dlnpsida))
    return a - gamma * dEda, dEda

In [23]:
N = 1000 #Number of walkers
T = 26000 #Number of displacement attempts per walker
T0 = 4000 #Reject first T0 timesteps from averaging

#Variational parameters
a = 0.1

d = 1/2 #mean step length
gamma = .5 #damped steepest descent parameter
epsilon = 1e-6 # tolerance on derivative 
itmax = 12; #max number of optimization iterations

dEda = 2; it = 0;
while((abs(dEda) >= epsilon) & (it<itmax)):
    r = np.random.uniform(-2*a,2*a,(N,6))
    R = np.empty((T*N,6))
    
    for t in range(0,T0):
        updateWalker(r,N,a)
        
    for t in range(0,T):
        updateWalker(r,N,a)
        R[t*N:(t+1)*N,:] = r
        
    Eloc = EL(R,a)
    E = np.mean(Eloc)
    VarE = np.var(Eloc)
    print('a = ',a,' <E> = ',E,' Var(EL) = ',VarE)
    a, dEda = updateVarParams(R,Eloc,a,gamma)
    it+=1
    


if it==itmax:
    print('Maximum number of iterations reached. No optimal variational parameter may be found up to the required precision')

a =  0.1  <E> =  -2.87701198739  Var(EL) =  0.136077566508
a =  0.157730818854  <E> =  -2.87790172659  Var(EL) =  0.108766536306
a =  0.144749894164  <E> =  -2.87776752303  Var(EL) =  0.114017088006
a =  0.143302737856  <E> =  -2.8781519172  Var(EL) =  0.114390947856
a =  0.143313972209  <E> =  -2.87780013341  Var(EL) =  0.114320869322
a =  0.143681523706  <E> =  -2.87796502674  Var(EL) =  0.114256096067
a =  0.142658382802  <E> =  -2.87774858248  Var(EL) =  0.11494798314
a =  0.142764044555  <E> =  -2.87816948875  Var(EL) =  0.114621421829
a =  0.143250304951  <E> =  -2.87787318286  Var(EL) =  0.114346205173
a =  0.143345012839  <E> =  -2.87812935635  Var(EL) =  0.114466356377
a =  0.142833752903  <E> =  -2.87819292081  Var(EL) =  0.114480600606
a =  0.14349770387  <E> =  -2.87848756762  Var(EL) =  0.114155865838
Maximum number of iterations reached. No optimal variational parameter may be found up to the required precision
